In [66]:
import numpy as np
from numpy import genfromtxt
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn.model_selection as sklearn
import torch.optim as optim

Load in the dataset. The 5 columns are `variance`, `skewness`, `curtosis`, `entropy`, and `class`.

In [241]:
banknotes = genfromtxt('banknote_auth.csv', delimiter=',')
banknotes = banknotes[1:]

In [242]:
np.random.shuffle(banknotes)
features = banknotes[:, 0:4]
classification = banknotes[:, 4]
classify = torch.Tensor(classification)
training_set = torch.utils.data.DataLoader(features, batch_size=8)

In [243]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 10)
        self.fc5 = nn.Linear(10, 10)
        self.fc6 = nn.Linear(10, 10)
        self.fc7 = nn.Linear(10, 10)
        self.fc8 = nn.Linear(10, 10)
        self.fc9 = nn.Linear(10, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = torch.tanh(self.fc5(x))
        x = torch.tanh(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = self.fc9(x)
        return F.log_softmax(x, dim=1)

In [244]:
net = Net()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [245]:
EPOCHS = 3

for epoch in range(EPOCHS):
    counter = 0
    loss = 0
    for X in training_set:
        #data is a batch of feature sets and labels
        #everytime you pass data through network, call net.zero_grad()
        net.zero_grad()
        output = net(X.view(-1, 4).float())
  
        loss = F.nll_loss(output, classify[counter:counter+8].long()) # negative log likelihood loss
        loss.backward() #backpropogate 
        optimizer.step() # adjusts weights for us
        counter += 8
    print(loss)

tensor(0.0363, grad_fn=<NllLossBackward>)
tensor(0.0029, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)


In [246]:
correct = 0
total = 0

with torch.no_grad(): #don't calculate gradients
    counter = 0
    for X in training_set:
        output = net(X.view(-1, 4).float())
        true_values = classify[counter:counter + 8].long()
        for idx, i in enumerate(output):
            if torch.argmax(i) == true_values[idx]:
                correct += 1
            total += 1
        counter += 8
            
print("Training Error: ", round((total - correct)/total, 3))

Training Error:  0.0


Perform K-Folds Cross Validation

In [247]:
validation_errors = []
kf = sklearn.KFold(n_splits =5)

np.random.shuffle(banknotes)
features = banknotes[:, 0:4]
classification = banknotes[:, 4]

for train_index, test_index in kf.split(features):

    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = classification[train_index], classification[test_index]
    
    net = Net()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    
    classify_train = torch.Tensor(y_train)
    training_set = torch.utils.data.DataLoader(X_train, batch_size=8)
    
    counter = 0
    for X in training_set:
        net.zero_grad()
        output = net(X.view(-1, 4).float())
  
        loss = F.nll_loss(output, classify_train[counter:counter+8].long())
        loss.backward()
        optimizer.step() 
        counter += 8
    
    val_training_set = torch.utils.data.DataLoader(X_test, batch_size=8)
    classify_test = torch.Tensor(y_test)
    with torch.no_grad(): #don't calculate gradients
        counter = 0
        total = 0
        correct = 0
        for X in val_training_set:
            output = net(X.view(-1, 4).float())
            true_values = classify_test[counter:counter + 8].long()
            for idx, i in enumerate(output):
                if torch.argmax(i) == true_values[idx]:
                    correct += 1
                total += 1
            counter += 8
        validation_errors.append((total - correct) / total)
    
print(validation_errors)
print("Validation Error: " + str(sum(validation_errors)/len(validation_errors)))    

[0.014545454545454545, 0.01090909090909091, 0.01824817518248175, 0.043795620437956206, 0.040145985401459854]
Validation Error: 0.025528865295288656
